This notebook is an optional extension to the first step in the workflow. 
In this case, I downloaded the US counties shapefile from the US Census Bureau.

The only inputs here are: 
* In path to your counties shapefile
* In path to your merged geotiff, which is the output of the previous script
* In county GEOID to filter the large dataframe by

I've resorted to using geopandas and rasterio here. They seem to be the lowest level libraries as possible without needing to circumvent much of vanilla GDAL's lengthy solutions.

In [1]:
import geopandas as gpd
import rasterio
import rasterio.mask

Inputs of the county shapefile, geotiff path, and geoid

In [2]:
in_shp = r"C:\Users\moult\Documents\geospatial\data\shapefiles\tl_2019_us_county.shp"
in_tif = r"C:\Users\moult\Documents\geospatial\data\Austin_Landsat.tif"
in_county_geoid = '48453'
counties_gdf = gpd.read_file(in_shp)
counties_gdf = counties_gdf[counties_gdf['GEOID'] == in_county_geoid]

This cell opens up the original geotiff, reprojects the dataframe to the appropriate 
UTM zone, and performs the mask in just a couple lines:

In [3]:
with rasterio.open(in_tif) as src:
    prj = src.crs.to_wkt()
    counties_gdf_reproj = counties_gdf.to_crs(prj)
    shapes = [counties_gdf_reproj['geometry'].values[0]]
    
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta

This final cell updates the output geotransformation with new dimensions
and then saves the clipped raster

In [4]:
out_meta.update({"driver":"GTiff",
                "height": out_image.shape[1],"width":out_image.shape[2],
                "transform": out_transform})

with rasterio.open(r"../../data/Austin_Landsat_Clip.tif", "w", **out_meta) as dest:
    dest.write(out_image)